In [1]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import knoxdata
from knoxdata.source.mpc import query_mpc_inventory
%matplotlib inline

# Scrape Data

In [2]:
import math
import json
# determined from website usage for bounds
knoxville_bounds = [
    [2509746.9416408655, 2659731.9416408655], # lat
    [567692.794594815, 639302.794594815], # long
]
query_area = 1000.0

latitudes = np.linspace(knoxville_bounds[0][0], knoxville_bounds[0][1], math.ceil((knoxville_bounds[0][1] - knoxville_bounds[0][0]) / query_area))
longitudes = np.linspace(knoxville_bounds[1][0], knoxville_bounds[1][1], math.ceil((knoxville_bounds[1][1] - knoxville_bounds[1][0]) / query_area))

print(len(latitudes), len(longitudes), 'total queries', len(latitudes) * len(longitudes))
# Caching will be very usefull (but request_cache doesn't work.......)

150 72 total queries 10800


In [3]:
import pickle
geo_data = pickle.load(open('latlong.pickle', 'rb'))
# geo_data = set()

In [4]:
len(geo_data)

100

In [5]:
import requests
import sqlite3
import time

start = time.time()
# raise NotImplementedError('API is down right now')
session = requests.session()
session.get('https://www.kgis.org/maps/MPCCases.html') # get cookies

conn = sqlite3.connect('../data/mpc/mpc.sqlite3')
lats = latitudes[10:40]
longs = longitudes[10:40]
for latitude in lats:
    for longitude in longs:
        if (latitude, longitude) in geo_data:
            continue
        query_mpc_inventory(latitude, longitude, query_area, session, conn)
        geo_data.add((latitude, longitude)) 
    
total_time = time.time() - start
print('DONNNNNEEEEE', total_time, 'seconds', total_time / (len(longs) * len(lats)))

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /proxy/proxy.ashx\non this server.<br />\n</p>\n</body></html>\n'
failed for url:  https://www.kgis.org/arcgis/rest/services/Maps/TreeInventory/MapServer/0/query?f
b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /proxy/proxy.ashx\non this server.<br />\n</p>\n</body></html>\n'
failed for url:  https://www.kgis.org/arcgis/rest/services/Maps/TreeInventory/MapServer/1/query?f
b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /proxy/proxy.ashx\non this server.<br />\n</p>\n</body></html>\n'
failed for url:  https://www.kgis.org/arcgis/rest/services/Maps/TreeInventory/MapServer

KeyboardInterrupt: 

In [17]:
import pickle
pickle.dump(geo_data, open('latlong.pickle', 'wb'))